In [1]:
from radarsimpy import Radar, Receiver, Transmitter
import plotly.graph_objs as go
from scipy import constants
from radarsimpy.simulator import sim_radar
import numpy as np

antenna_gain = 20  # dBi

az_angle = np.arange(-20, 21, 1)
az_pattern = 20 * np.log10(np.cos(az_angle / 180 * np.pi) ** 500) + antenna_gain

el_angle = np.arange(-20, 21, 1)
el_pattern = 20 * np.log10((np.cos(el_angle / 180 * np.pi)) ** 400) + antenna_gain

light_speed = constants.c

max_range = 5000  # Maximum unambiguous range
range_res = 500  # Required range resolution
pulse_bw = light_speed / (2 * range_res)  # Pulse bandwidth
pulse_width = 1 / pulse_bw  # Pulse width
prf = light_speed / (2 * max_range)  # Pulse repetition frequency
prp = 1 / prf
fs = 64e6  # Sampling rate 6 msps
fc = 35e9  # Carrier frequency
num_pulse = 1

total_samples = int(prp * num_pulse * fs)
sample_per_pulse = int(total_samples / num_pulse)

mod_t = np.arange(0, total_samples) / fs  # Time series of the modulation
amp = np.zeros_like(mod_t)  # Amplitude of the modulation
amp[mod_t <= pulse_width] = 1  # Create a rectangular pulse

tx_channel = dict(
    location=(0, 0, 0),
    azimuth_angle=az_angle,
    azimuth_pattern=az_pattern,
    elevation_angle=el_angle,
    elevation_pattern=el_pattern,
    amp=amp,
    mod_t=mod_t,
)

tx = Transmitter(
    f=fc,
    t=1 / prf,
    tx_power=67,  # dBm
    pulses=num_pulse,
    channels=[tx_channel],
)

rx_channel = dict(
    location=(0, 0, 0),
    azimuth_angle=az_angle,
    azimuth_pattern=az_pattern,
    elevation_angle=el_angle,
    elevation_pattern=el_pattern,
)

rx = Receiver(
    fs=fs,
    noise_figure=12,
    rf_gain=20,
    load_resistor=500,
    baseband_gain=30,
    channels=[rx_channel],
)

radar = Radar(transmitter=tx, receiver=rx)

target = dict(location=(1000, 0, 0), speed=(150, 0, 0), rcs=10, phase=0)

data = sim_radar(radar, [target])
timestamp = data["timestamp"]
baseband = data["baseband"]  # noise is left out on purpose

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=timestamp[0, 0, :],
        y=np.arctan2(np.real(baseband[0, 0, :]), np.imag(baseband[0, 0, :])),
        name="Angle",
    )
)
fig.update_layout(
    yaxis=dict(title="Angle (rad)"),
    xaxis=dict(title="Time (s)"),
    template="plotly_dark",
)
fig.show()

In [2]:
tx_channel = {
    "location": (0, 0, 0),
}

freq_nonlinear = np.array(
    [
        2.40750000e10,
        2.40760901e10,
        2.40771786e10,
        2.40782654e10,
        2.40793506e10,
        2.40804341e10,
        2.40815161e10,
        2.40825964e10,
        2.40836750e10,
        2.40847521e10,
        2.40858275e10,
        2.40869012e10,
        2.40879734e10,
        2.40890439e10,
        2.40901127e10,
        2.40911800e10,
        2.40922456e10,
        2.40933096e10,
        2.40943719e10,
        2.40954326e10,
        2.40964917e10,
        2.40975491e10,
        2.40986049e10,
        2.40996591e10,
        2.41007117e10,
        2.41017626e10,
        2.41028119e10,
        2.41038595e10,
        2.41049055e10,
        2.41059499e10,
        2.41069927e10,
        2.41080338e10,
        2.41090733e10,
        2.41101111e10,
        2.41111473e10,
        2.41121819e10,
        2.41132149e10,
        2.41142462e10,
        2.41152759e10,
        2.41163039e10,
        2.41173304e10,
        2.41183552e10,
        2.41193783e10,
        2.41203999e10,
        2.41214198e10,
        2.41224380e10,
        2.41234546e10,
        2.41244696e10,
        2.41254830e10,
        2.41264947e10,
        2.41275048e10,
        2.41285133e10,
        2.41295202e10,
        2.41305254e10,
        2.41315289e10,
        2.41325309e10,
        2.41335312e10,
        2.41345298e10,
        2.41355269e10,
        2.41365223e10,
        2.41375161e10,
        2.41385082e10,
        2.41394987e10,
        2.41404876e10,
        2.41414748e10,
        2.41424605e10,
        2.41434444e10,
        2.41444268e10,
        2.41454075e10,
        2.41463866e10,
        2.41473640e10,
        2.41483399e10,
        2.41493140e10,
        2.41502866e10,
        2.41512575e10,
        2.41522268e10,
        2.41531945e10,
        2.41541605e10,
        2.41551249e10,
        2.41560876e10,
        2.41570488e10,
        2.41580083e10,
        2.41589661e10,
        2.41599224e10,
        2.41608770e10,
        2.41618299e10,
        2.41627812e10,
        2.41637309e10,
        2.41646790e10,
        2.41656254e10,
        2.41665702e10,
        2.41675134e10,
        2.41684550e10,
        2.41693949e10,
        2.41703331e10,
        2.41712698e10,
        2.41722048e10,
        2.41731381e10,
        2.41740699e10,
        2.41750000e10,
    ]
)

tx_nonlinear = Transmitter(
    f=freq_nonlinear,
    t=np.linspace(0, 80e-6, 100),
    tx_power=40,
    prp=100e-6,
    pulses=1,
    channels=[tx_channel],
)

rx_channel = {
    "location": (0, 0, 0),
}

rx = Receiver(
    fs=2e5,
    noise_figure=12,
    rf_gain=40,
    load_resistor=500,
    baseband_gain=30,
    channels=[rx_channel],
)

radar_nonlinear = Radar(transmitter=tx_nonlinear, receiver=rx)

targets = [
    {
        "model": "../models/cr.stl",
        "location": (200, 0, 0),
    }
]

result = sim_radar(radar_nonlinear, targets, density=1)

In [3]:
result["baseband"]

array([[[ 0.00042601-2.93681651e-03j,  0.00148935-2.56677427e-03j,
         -0.00046196-2.93141011e-03j, -0.00290341-6.13889558e-04j,
         -0.00014568+2.96402297e-03j,  0.00239102-1.75774519e-03j,
         -0.00287518+7.34832136e-04j,  0.00272075-1.18493486e-03j,
         -0.00135757+2.63884542e-03j, -0.0019856 -2.20539896e-03j,
          0.00185658-2.31502894e-03j,  0.00296738-2.79537656e-05j,
          0.0027857 +1.02265165e-03j,  0.00291446+5.58266724e-04j,
          0.00260711-1.41722117e-03j, -0.00040577-2.93949713e-03j]]])